## RUNNING INFERENCE

In [1]:
%matplotlib inline

from torch.optim import Adam
import torch
from code.model import UNetWrapper


In [ ]:
device = torch.device("cpu")

model = UNetWrapper(
    out_channels=1, 
    in_channels=1,
)

model = model.to(device)
PATH="TFM_FINAL_3D\\data-unversioned\\models\\p2ch13\\seg_2023-06-11_18.04.40_final_seg.8008.state"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state'])

model.eval()


In [ ]:
import glob
import os
import nibabel as nib
from scipy import ndimage
import numpy as np

def xyz2zxy(X):
    return np.transpose(X, (2, 0, 1))

path = "Here gpoes you input folder containing teh files"
mean_dice=0
for file in os.listdir(path):
    nii_path = glob.glob(
        f"{path}\\{file}\\TOF_{file}_0000.nii.gz"
    )
    mra_nii = nib.load(nii_path[0])
    mra_a = np.array(mra_nii.get_fdata(), dtype=np.float32) 
    mra_a=xyz2zxy(mra_a)

    nii_path = glob.glob(
        f"{path}\\{file}\\TOF_{file}.nii.gz"
    )
    mra_nii = nib.load(nii_path[0])
    pos_a = np.array(mra_nii.get_fdata(), dtype=np.float32) 
    pos_a=xyz2zxy(pos_a)

    lbl = ndimage.label(pos_a)[0]
    labels = list(np.unique(lbl))
    labels = [x for x in range(1,len(labels))]
    center_of_mass = ndimage.center_of_mass(pos_a, lbl,[labels])

    pred = np.zeros(mra_a.shape)
    for element in center_of_mass:
        if mra_a[int(element[0][0]),int(element[0][1]),int(element[0][2])]>0.306*np.max(mra_a):
            z_min=int(element[0][0])-8
            z_max=int(element[0][0])+8
            x_min=int(element[0][1])-32
            x_max=int(element[0][1])+32
            y_min=int(element[0][2])-32
            y_max=int(element[0][2])+32

            z_min = max([z_min, 0]) if max([element[0][0] - 8, 0]) == 0 else z_min
            z_max = min([z_max, pos_a.shape[0]]) if min([element[0][0] + 8, pos_a.shape[0]]) == pos_a.shape[0] else z_max
            x_min = max([x_min, 0]) if max([element[0][1] - 32, 0]) == 0 else x_min
            x_max = min([x_max, pos_a.shape[1]]) if min([element[0][1] + 32, pos_a.shape[1]]) == pos_a.shape[1] else x_max
            y_min = max([y_min, 0]) if max([element[0][2] - 32, 0]) == 0 else y_min
            y_max = min([y_max, pos_a.shape[2]]) if min([element[0][2] + 32, pos_a.shape[2]]) == pos_a.shape[2] else y_max


            input = mra_a[z_min:z_max,x_min:x_max,y_min:y_max]
            input = torch.from_numpy(input)
            output = model(input.unsqueeze(0).unsqueeze(0))[0][0]
            final = output.detach().cpu().numpy()>0.5
            pred[z_min:z_max,x_min:x_max,y_min:y_max]+=final

    pred = np.clip(pred, 0, 1)
    transposed_pred = np.transpose(pred, (1, 2, 0))
    final_img = nib.Nifti1Image(transposed_pred, mra_nii.affine)
    nib.save(final_img, f"{path}\\{file}\\TOF_{file}_pred.nii.gz")
